In [1]:
import datetime
from pyspark.sql import DataFrame, Row
from pyspark.sql.types import *
from pyspark.sql.functions import unix_timestamp, from_unixtime, col
from pyspark.sql import functions as F
import sys
from hops import hdfs as hdfs
print(hdfs.project_path())

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
19,application_1613204927129_0028,pyspark,idle,Link,Link


SparkSession available as 'spark'.
hdfs://rpc.namenode.service.consul:8020/Projects/pittre/

In [2]:
fg1_schema = StructType([
  StructField("id", IntegerType(), True),
  StructField("ts", IntegerType(), True),
  StructField("f1", StringType(), True)    
])

fg1=spark.read.csv("hdfs:///Projects/" + hdfs.project_name() + "/Resources/1000000-20-2-out.csv", header=True, schema=fg1_schema)
fg1=fg1.sort(col("id"),col("ts"))
fg1.show()

+---+----+-----+
| id|  ts|   f1|
+---+----+-----+
|  1|1002| f2-1|
|  1|1004| f2-2|
|  1|1006| f2-3|
|  1|1008| f2-4|
|  1|1010| f2-5|
|  1|1012| f2-6|
|  1|1014| f2-7|
|  1|1016| f2-8|
|  1|1018| f2-9|
|  1|1020|f2-10|
|  2|1002|f2-11|
|  2|1004|f2-12|
|  2|1006|f2-13|
|  2|1008|f2-14|
|  2|1010|f2-15|
|  2|1012|f2-16|
|  2|1014|f2-17|
|  2|1016|f2-18|
|  2|1018|f2-19|
|  2|1020|f2-20|
+---+----+-----+
only showing top 20 rows

In [3]:
fg2_schema = StructType([
  StructField("id", IntegerType(), True),
  StructField("ts", IntegerType(), True),
  StructField("f2", StringType(), True)    
])

fg2=spark.read.csv("hdfs:///Projects/" + hdfs.project_name() + "/Resources/1000000-20-2-out.csv", header=True, schema=fg2_schema)
fg2.show()

+---+----+-----+
| id|  ts|   f2|
+---+----+-----+
|  1|1002| f2-1|
|  1|1004| f2-2|
|  1|1006| f2-3|
|  1|1008| f2-4|
|  1|1010| f2-5|
|  1|1012| f2-6|
|  1|1014| f2-7|
|  1|1016| f2-8|
|  1|1018| f2-9|
|  1|1020|f2-10|
|  2|1002|f2-11|
|  2|1004|f2-12|
|  2|1006|f2-13|
|  2|1008|f2-14|
|  2|1010|f2-15|
|  2|1012|f2-16|
|  2|1014|f2-17|
|  2|1016|f2-18|
|  2|1018|f2-19|
|  2|1020|f2-20|
+---+----+-----+
only showing top 20 rows

In [ ]:
fg1a=fg1.withColumnRenamed("id", "id_1").withColumnRenamed("ts", "ts_1")
joined=fg1a.join(fg2, (fg1a.id_1 == fg2.id) & (fg1a.ts_1 == fg2.ts), how="full_outer").orderBy(fg1a.id_1, fg1a.ts_1)\
.withColumnRenamed("ts", "ts_2").withColumnRenamed("id", "id_2")
joined.show()

In [ ]:
from pyspark.sql import functions as F
joined2 = joined.withColumn('id',F.when(joined.id_1.isNull(),joined.id_2).otherwise(joined.id_1)).drop(joined.id_1).drop(joined.id_2).na.fill(0).na.fill("")
joined2.orderBy(joined2.id).show()

In [6]:
from pyspark.sql import functions as F

grouped_rdd = joined2.groupBy(joined2.id).agg(F.collect_list(joined2.ts_1), F.collect_list(joined2.f1), F.collect_list(joined2.ts_2), F.collect_list(joined2.f2))
grouped_rdd.collect()

[Row(id=1, collect_list(ts_1)=[0, 0, 1002, 1004, 1006, 1008, 1010], collect_list(f1)=['', '', 'f2-1', 'f2-2', 'f2-3', 'f2-4', 'f2-5'], collect_list(ts_2)=[1003, 1009, 0, 0, 1006, 0, 0], collect_list(f2)=['f3-1', 'f3-3', '', '', 'f3-2', '', '']), Row(id=3, collect_list(ts_1)=[0, 0, 1002, 1004, 1006, 1008, 1010], collect_list(f1)=['', '', 'f2-11', 'f2-12', 'f2-13', 'f2-14', 'f2-15'], collect_list(ts_2)=[1003, 1009, 0, 0, 1006, 0, 0], collect_list(f2)=['f3-7', 'f3-9', '', '', 'f3-8', '', '']), Row(id=5, collect_list(ts_1)=[0, 0, 1002, 1004, 1006, 1008, 1010], collect_list(f1)=['', '', 'f2-21', 'f2-22', 'f2-23', 'f2-24', 'f2-25'], collect_list(ts_2)=[1009, 1003, 0, 0, 1006, 0, 0], collect_list(f2)=['f3-15', 'f3-13', '', '', 'f3-14', '', '']), Row(id=4, collect_list(ts_1)=[0, 0, 1002, 1004, 1006, 1008, 1010], collect_list(f1)=['', '', 'f2-16', 'f2-17', 'f2-18', 'f2-19', 'f2-20'], collect_list(ts_2)=[1009, 1003, 0, 0, 1006, 0, 0], collect_list(f2)=['f3-12', 'f3-10', '', '', 'f3-11', '', ''])

In [7]:
from collections import OrderedDict 

def align(ts_1, f1, ts_2, f2):
    output={}
    
    ts2_dict={}
    for i in range(len(ts_2)):
        if ts_2[i] != 0:
            ts2_dict[ts_2[i]]=f2[i]
    ts2_dict = OrderedDict(sorted(ts2_dict.items()))     

    ts1_dict = {}
    for i in range(len(ts_1)):
        if ts_1[i] != 0:
            ts1_dict[ts_1[i]]=f1[i]
    ts1_dict = OrderedDict(sorted(ts1_dict.items()))     

    # sort the ts2 keys by value, just in case
    ts2_keys = sorted(ts2_dict.keys())
    ts1_keys = sorted(ts1_dict.keys())
            
    k=0
    for i in range(len(ts1_keys)):
            found=False
            # need to search from beginning to the current position, and from the current position forward
            # Optimization possible here. Start at the key value for the most recently seen value of this feature.
            while ts1_keys[i] >= ts2_keys[k] :
                found=True
                if len(ts2_keys) <= k+1:
                    break
                if ts1_keys[i] >= ts2_keys[k+1]:
                    k+=1
                else:
                    break
            # at this point, we have the closest key to ts_1[i]
            # add to dict all the features f1, f2
            if found==True:
                output[ts1_keys[i]]=(ts1_dict[ts1_keys[i]], ts2_dict[ts2_keys[k]])
    return output
        

In [8]:
my_rdd = grouped_rdd.rdd.map(lambda x: (x[0],align(x[1],x[2],x[3],x[4])))
my_rdd.collect()

[(1, {1004: ('f2-2', 'f3-1'), 1006: ('f2-3', 'f3-2'), 1008: ('f2-4', 'f3-2'), 1010: ('f2-5', 'f3-3')}), (3, {1004: ('f2-12', 'f3-7'), 1006: ('f2-13', 'f3-8'), 1008: ('f2-14', 'f3-8'), 1010: ('f2-15', 'f3-9')}), (5, {1004: ('f2-22', 'f3-13'), 1006: ('f2-23', 'f3-14'), 1008: ('f2-24', 'f3-14'), 1010: ('f2-25', 'f3-15')}), (4, {1004: ('f2-17', 'f3-10'), 1006: ('f2-18', 'f3-11'), 1008: ('f2-19', 'f3-11'), 1010: ('f2-20', 'f3-12')}), (2, {1004: ('f2-7', 'f3-4'), 1006: ('f2-8', 'f3-5'), 1008: ('f2-9', 'f3-5'), 1010: ('f2-10', 'f3-6')})]

In [9]:
schema=["id", "values"]
df=my_rdd.toDF(schema)
df.show()

+---+--------------------+
| id|              values|
+---+--------------------+
|  1|[1008 -> [f2-4, f...|
|  3|[1008 -> [f2-14, ...|
|  5|[1008 -> [f2-24, ...|
|  4|[1008 -> [f2-19, ...|
|  2|[1008 -> [f2-9, f...|
+---+--------------------+

In [10]:
df2 = df.select(df.id,F.explode(df.values)).orderBy(df.id)
df3=df2.orderBy(df2.id, df2.key)
df3.show()

+---+----+--------------+
| id| key|         value|
+---+----+--------------+
|  1|1004|  [f2-2, f3-1]|
|  1|1006|  [f2-3, f3-2]|
|  1|1008|  [f2-4, f3-2]|
|  1|1010|  [f2-5, f3-3]|
|  2|1004|  [f2-7, f3-4]|
|  2|1006|  [f2-8, f3-5]|
|  2|1008|  [f2-9, f3-5]|
|  2|1010| [f2-10, f3-6]|
|  3|1004| [f2-12, f3-7]|
|  3|1006| [f2-13, f3-8]|
|  3|1008| [f2-14, f3-8]|
|  3|1010| [f2-15, f3-9]|
|  4|1004|[f2-17, f3-10]|
|  4|1006|[f2-18, f3-11]|
|  4|1008|[f2-19, f3-11]|
|  4|1010|[f2-20, f3-12]|
|  5|1004|[f2-22, f3-13]|
|  5|1006|[f2-23, f3-14]|
|  5|1008|[f2-24, f3-14]|
|  5|1010|[f2-25, f3-15]|
+---+----+--------------+

In [11]:
df4=df3.select("value.*", "*").drop(df3.value)
df4=df4.withColumnRenamed("_1", "f1").withColumnRenamed("_2","f2").withColumnRenamed("key","ts")
df4.show()

+-----+-----+---+----+
|   f1|   f2| id|  ts|
+-----+-----+---+----+
| f2-2| f3-1|  1|1004|
| f2-3| f3-2|  1|1006|
| f2-4| f3-2|  1|1008|
| f2-5| f3-3|  1|1010|
| f2-7| f3-4|  2|1004|
| f2-8| f3-5|  2|1006|
| f2-9| f3-5|  2|1008|
|f2-10| f3-6|  2|1010|
|f2-12| f3-7|  3|1004|
|f2-13| f3-8|  3|1006|
|f2-14| f3-8|  3|1008|
|f2-15| f3-9|  3|1010|
|f2-17|f3-10|  4|1004|
|f2-18|f3-11|  4|1006|
|f2-19|f3-11|  4|1008|
|f2-20|f3-12|  4|1010|
|f2-22|f3-13|  5|1004|
|f2-23|f3-14|  5|1006|
|f2-24|f3-14|  5|1008|
|f2-25|f3-15|  5|1010|
+-----+-----+---+----+

In [12]:
df5=df4.select("id","ts","f1","f2")
df5.dtypes

[('id', 'bigint'), ('ts', 'bigint'), ('f1', 'string'), ('f2', 'string')]

In [13]:
df5.show()

+---+----+-----+-----+
| id|  ts|   f1|   f2|
+---+----+-----+-----+
|  1|1004| f2-2| f3-1|
|  1|1006| f2-3| f3-2|
|  1|1008| f2-4| f3-2|
|  1|1010| f2-5| f3-3|
|  2|1004| f2-7| f3-4|
|  2|1006| f2-8| f3-5|
|  2|1008| f2-9| f3-5|
|  2|1010|f2-10| f3-6|
|  3|1004|f2-12| f3-7|
|  3|1006|f2-13| f3-8|
|  3|1008|f2-14| f3-8|
|  3|1010|f2-15| f3-9|
|  4|1004|f2-17|f3-10|
|  4|1006|f2-18|f3-11|
|  4|1008|f2-19|f3-11|
|  4|1010|f2-20|f3-12|
|  5|1004|f2-22|f3-13|
|  5|1006|f2-23|f3-14|
|  5|1008|f2-24|f3-14|
|  5|1010|f2-25|f3-15|
+---+----+-----+-----+